In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

# Assuming your data files are saved in "(your current directory)/io/" folder. 

In [2]:
train = pd.read_csv("train.csv")
train.head()

,Donation ID,Donor ID,Donation Amount,Donation Received Date,# of Donations Made,min,max,Total Sum Donated,Donor State,Donor Is Teacher,...,School State,# Donations Received,Total Donation Amt Received,Teacher Project Posted Sequence,Project Title,Project Subject Category Tree,Project Subject Subcategory Tree,Project Resource Category,Project Current Status,Resource Vendor Name
0,579821d4de7922dd32ae6aa9f1e67fd9,00070fd37d3e8b5b7c08a4a0ba22aba1,50.00,2016-01-18 18:40:29,29,10.00,150.0,1560.00,Oklahoma,No,...,Oklahoma,8,375.00,5,Science Lab Supplies,Math & Science,"Applied Sciences, Health & Life Science",Supplies,Expired,"Carolina Biological Supply Company,Carolina Bi..."
1,f2bbdba4619801071b2e97a483ff2c71,00070fd37d3e8b5b7c08a4a0ba22aba1,50.00,2017-09-17 12:46:24,29,10.00,150.0,1560.00,Oklahoma,No,...,Oklahoma,6,370.13,2,Your Epidermis is Showing...Your Art!,Music & The Arts,Visual Arts,Supplies,Fully Funded,"Amazon Business,Amazon Business"
2,886fdf2feda0118c8f2adcac7b15b0e4,0015dc157335e5ad6f569e5264bbf33e,50.00,2016-01-10 10:11:22,22,10.00,2700.0,4505.86,Maryland,Yes,...,Maryland,5,718.28,20,Shine Some Light on the Subject!,"Special Needs, Music & The Arts","Special Needs, Visual Arts",Supplies,Fully Funded,"Amazon Business,Amazon Business,Amazon Busines..."
3,02216d6d22189b292fd6285285b95c12,0015dc157335e5ad6f569e5264bbf33e,41.00,2013-04-24 16:23:11,22,10.00,2700.0,4505.86,Maryland,Yes,...,Maryland,2,143.16,11,Repetition With Excitement!,"Applied Learning, Special Needs","Early Development, Special Needs",Books,Fully Funded,"AKJ Education,AKJ Education,AKJ Education,AKJ ..."
4,7798f746c2956bfd29f394b5b127e6a4,00199e3565635f8a5ebefd3b5985a7f3,238.65,2018-01-25 08:57:10,215,11.24,1116.4,46502.98,Massachusetts,No,...,Massachusetts,1,238.65,3,Teaching Kids Sports,Health & Sports,"Health & Wellness, Team Sports",Sports & Exercise Equipment,Fully Funded,"School Specialty,Staples Advantage,School Spec..."


## Cosine Similarity 

In [3]:
# list1v = train.columns.values.tolist()
# print(list1v)

cosimilar_df = train[['Donor ID', 'Project ID','Donation ID','Donation Amount','Project Category- Applied Learning', 'Project Category- Health & Sports', 'Project Category- History & Civics', 'Project Category- Literacy & Language', 'Project Category- Math & Science', 'Project Category- Music & The Arts','Project Category- Special Needs', 'Project Category- Warmth, Care & Hunger']]
cosimilar_df.head()

import math

def smooth_donor_preference(x):
    return math.log(1+x, 2)
    
cosine_df_donatestrength = cosimilar_df \
                    .groupby(['Donor ID', 'Project ID','Donation ID'])['Donation Amount'].sum() \
                    .apply(smooth_donor_preference).reset_index()

cosine_df_donatestrength.head()
cosine_df_donatestrength.columns = ['Donor ID', 'Project ID','Donation ID','Donation Strength']
cosine_df1 = pd.merge(cosimilar_df, right=cosine_df_donatestrength, on=["Donor ID","Project ID",'Donation ID'], how="inner")
variables_drop = ['Donation Amount','Project ID','Donation ID']
cosine_df2 = cosine_df1[cosine_df1.columns.drop(variables_drop)]
variable_df_multiple = cosine_df2[['Project Category- Applied Learning', 'Project Category- Health & Sports', 'Project Category- History & Civics', 'Project Category- Literacy & Language', 'Project Category- Math & Science', 'Project Category- Music & The Arts','Project Category- Special Needs', 'Project Category- Warmth, Care & Hunger']].multiply(cosine_df2['Donation Strength'], axis="index")
cosine_df3 = pd.concat([cosine_df2['Donor ID'],variable_df_multiple],axis=1)
cosine_df4 = cosine_df3.groupby(['Donor ID']).sum().reset_index()
cosine_df5 = cosine_df4[cosine_df4.columns.drop(['Donor ID'])]
cosine_df5.head()

,Project Category- Applied Learning,Project Category- Health & Sports,Project Category- History & Civics,Project Category- Literacy & Language,Project Category- Math & Science,Project Category- Music & The Arts,Project Category- Special Needs,"Project Category- Warmth, Care & Hunger"
0,0.000000,0.000000,0.0,0.000000,5.672425,5.672425,0.000000,0.0
1,5.392317,0.000000,0.0,0.000000,0.000000,5.672425,11.064743,0.0
2,0.000000,39.638098,0.0,0.000000,0.000000,0.000000,0.000000,0.0
3,0.000000,0.000000,0.0,5.672425,13.316423,6.658211,6.658211,0.0
4,0.000000,0.000000,0.0,5.392317,5.392317,0.000000,0.000000,0.0


In [4]:
from scipy import sparse
cosine_df_sparse = sparse.csr_matrix(cosine_df5)
# print(cosine_df_sparse)

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_df_similarities = cosine_similarity(cosine_df_sparse)
cosine_df_similarities_sparse = sparse.csr_matrix(cosine_df_similarities)
print(cosine_df_similarities_sparse)

  (0, 0)	0.9999999999999998
  (0, 1)	0.29595143959370823
  (0, 3)	0.8179636209368781
  (0, 4)	0.5
  (0, 5)	0.8164965809277259
  (0, 6)	0.36433168221519663
  (0, 7)	0.43115228710352826
  (0, 8)	0.6097558353757594
  (0, 9)	0.3788422613828998
  (0, 10)	0.7071067811865475
  (0, 11)	0.8597258107417951
  (0, 12)	0.12684251632611132
  (0, 13)	0.5418100575933782
  (0, 15)	0.793960914873322
  (0, 16)	0.3214501838443423
  (0, 19)	0.2981873551110751
  (0, 20)	0.31622776601683783
  (0, 21)	0.39871206349823296
  (0, 23)	0.43832322011685226
  (0, 24)	0.17509309524979164
  (0, 26)	0.5428179529791463
  (0, 28)	0.8017837257372731
  (0, 29)	0.6743802172901854
  (0, 30)	0.408248290463863
  (0, 31)	0.5217016862479367
  :	:
  (8784, 8746)	0.5
  (8784, 8747)	0.5486826666714103
  (8784, 8748)	0.605692619619863
  (8784, 8753)	0.40910914387809466
  (8784, 8754)	0.22614876586156765
  (8784, 8755)	0.9214079835812045
  (8784, 8757)	0.8403974994111616
  (8784, 8759)	0.6702050965242221
  (8784, 8760)	0.972816944257

In [6]:
donorsim_df1 = pd.DataFrame(cosine_df_similarities_sparse.todense())
donorsim_df1['Donor'] = donorsim_df1.index
donorsim_df1.head()

,0,1,2,3,4,5,6,7,8,9,...,8776,8777,8778,8779,8780,8781,8782,8783,8784,Donor
0,1.000000,0.295951,0.0,0.817964,0.500000,0.816497,0.364332,0.431152,0.609756,0.378842,...,0.211343,0.394591,0.295369,0.437516,0.000000,0.499471,0.500000,0.000000,0.707107,0
1,0.295951,1.000000,0.0,0.476186,0.000000,0.241643,0.488140,0.000000,0.360916,0.191842,...,0.125095,0.455585,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
2,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.686562,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.723256,0.000000,0.000000,0.000000,0.000000,2
3,0.817964,0.476186,0.0,1.000000,0.777596,0.857526,0.367932,0.550307,0.498844,0.770199,...,0.428734,0.847275,0.620605,0.735233,0.226857,0.544732,0.777596,0.290721,0.771183,3
4,0.500000,0.000000,0.0,0.777596,1.000000,0.816497,0.364332,0.603535,0.358047,0.952314,...,0.674784,0.828868,0.937831,0.993016,0.488314,0.499471,1.000000,0.625781,0.707107,4


In [7]:
donor_namelist = cosine_df4[['Donor ID']]
donor_namelist['Donor'] = donor_namelist.index + 1
donor_namelist.head()

c:\users\me\desktop\pythonapp\testprojs\venv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Donor ID,Donor
0,00070fd37d3e8b5b7c08a4a0ba22aba1,1
1,0015dc157335e5ad6f569e5264bbf33e,2
2,00199e3565635f8a5ebefd3b5985a7f3,3
3,00200ccc822140acfc2f15dc809d15e5,4
4,0026a8fc3a4faea82ccab26c6f55b709,5


In [8]:
donorsim_df2 = pd.melt(donorsim_df1,id_vars='Donor')
donorsim_df2.Donor = donorsim_df2.Donor + 1
donorsim_df2.variable = donorsim_df2.variable + 1
donorsim_df3 = donorsim_df2[donorsim_df2.value != 0]
donorsim_df4 = donorsim_df3[donorsim_df3['variable'] != donorsim_df3['Donor']]
donorsim_df5 = pd.merge(left=donorsim_df4, right=donor_namelist, on="Donor", how="inner")
donorsim_df6 = donorsim_df5[donorsim_df5.columns.drop(['Donor'])]
donorsim_df6.columns = ['Donor', 'Similarity Score', 'Original Donor ID']
donorsim_df7 = pd.concat([donorsim_df6,donor_namelist], axis=1, sort=False)
cols = [0,4]
donorsim_df7.drop(donorsim_df7.columns[cols],axis=1,inplace=True)
donorsim_df7.columns = ['Similarity Score', 'Original Donor ID','Other Donor ID']
donorsim_df7.head()

,Similarity Score,Original Donor ID,Other Donor ID
0,0.295951,0015dc157335e5ad6f569e5264bbf33e,00070fd37d3e8b5b7c08a4a0ba22aba1
1,0.476186,0015dc157335e5ad6f569e5264bbf33e,0015dc157335e5ad6f569e5264bbf33e
2,0.241643,0015dc157335e5ad6f569e5264bbf33e,00199e3565635f8a5ebefd3b5985a7f3
3,0.488140,0015dc157335e5ad6f569e5264bbf33e,00200ccc822140acfc2f15dc809d15e5
4,0.360916,0015dc157335e5ad6f569e5264bbf33e,0026a8fc3a4faea82ccab26c6f55b709


In [9]:
donorproj_df1 = cosine_df1.groupby(['Donor ID','Project ID'])['Donation Amount'].agg(['count'])
donorproj_df2 = donorproj_df1[donorproj_df1.columns.drop(['count'])]
donorproj_df3 = donorproj_df2.reset_index()
donorproj_df3.tail()

,Donor ID,Project ID
34689,fff99b7b56af3835b317bc1233bb9595,1e9b3f44be493f155067e8261fcd34ef
34690,fff99b7b56af3835b317bc1233bb9595,5a4a394d00fb96df0bc43006d3a2151c
34691,fff99b7b56af3835b317bc1233bb9595,6820b430838c5dc51690a4d538d25749
34692,fffd69c0f1dc11f7fc10d595891723a2,4eb5d72bfd1e1577536267269e7aee19
34693,fffd69c0f1dc11f7fc10d595891723a2,81426170e4ee9b1d27b8316a95b93703


In [10]:
# donorsim_df7.to_csv('donorsim.csv', encoding='utf-8', index=False)
# donorproj_df1.to_csv('donorproj.csv', encoding='utf-8', index=False)

In [11]:
test = pd.read_csv("test.csv")
testset = test.groupby(['Donor ID','Project ID'])['Donation Amount'].agg(['count'])
testset2 = testset[testset.columns.drop(['count'])]
testset3 = testset2.reset_index()
testset3.head()

,Donor ID,Project ID
0,006c2d57cca011d6040fed3a2ceb8179,dd19fb9ec77a69b6e9ffd956a3a88b5b
1,009d5fc7b87883ffad248db5150bf1fc,0c7d99a1079bb22a500e71a7647a54b6
2,009d5fc7b87883ffad248db5150bf1fc,13ffe6f614ee069f1a6faa303bd5a94e
3,009d5fc7b87883ffad248db5150bf1fc,958909abb8551c487f34570b2ded43d6
4,009d5fc7b87883ffad248db5150bf1fc,c5ce8362cc1ea4ec1f99ba880252de63


In [12]:
threshold = donorsim_df7["Similarity Score"].median()
print(threshold)

0.6075266070122004


In [13]:
list_of_donor_ids = testset3['Donor ID'].values.tolist()
print(len(list_of_donor_ids))
# print(list_of_donor_ids)
#score set

1248


In [14]:
scoreset = donorsim_df7.set_index('Other Donor ID').copy()
scoreset.head()

,Similarity Score,Original Donor ID
Other Donor ID,,
00070fd37d3e8b5b7c08a4a0ba22aba1,0.295951,0015dc157335e5ad6f569e5264bbf33e
0015dc157335e5ad6f569e5264bbf33e,0.476186,0015dc157335e5ad6f569e5264bbf33e
00199e3565635f8a5ebefd3b5985a7f3,0.241643,0015dc157335e5ad6f569e5264bbf33e
00200ccc822140acfc2f15dc809d15e5,0.488140,0015dc157335e5ad6f569e5264bbf33e
0026a8fc3a4faea82ccab26c6f55b709,0.360916,0015dc157335e5ad6f569e5264bbf33e


In [15]:
check_df = scoreset.loc[list_of_donor_ids].copy()
check_df.describe()
check_df.head()

,Similarity Score,Original Donor ID
Other Donor ID,,
006c2d57cca011d6040fed3a2ceb8179,0.462709,0015dc157335e5ad6f569e5264bbf33e
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e


In [16]:
joined = check_df.join(donorproj_df3.set_index('Donor ID'))
joined.describe()
joined.head()

,Similarity Score,Original Donor ID,Project ID
006c2d57cca011d6040fed3a2ceb8179,0.462709,0015dc157335e5ad6f569e5264bbf33e,dd19fb9ec77a69b6e9ffd956a3a88b5b
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,022a1517e63f4e66845358f15ff7793e
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,02f2f8fc72b4714f5c6b98c04edb5913
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,0378b6a177fd71d4416b764538b9daa8
009d5fc7b87883ffad248db5150bf1fc,0.379582,0015dc157335e5ad6f569e5264bbf33e,07c5fc3f0b880304cefb39b3c43a34ed


In [17]:
joined[joined.isnull().any(axis=1)]

,Similarity Score,Original Donor ID,Project ID


In [18]:
projectitle = train[['Project ID','Project Title']]
projectitle.head()

,Project ID,Project Title
0,857f4517b5e2aacf6899e489936ec5a8,Science Lab Supplies
1,eddfdd15f9c22fe23b306b7f54918c81,Your Epidermis is Showing...Your Art!
2,52f352059701bb7c54990de75f9c108d,Shine Some Light on the Subject!
3,ded87d79ea309cb47686ed54bf4f3f2d,Repetition With Excitement!
4,3faed6ae4805456447f2b9f22cf0c950,Teaching Kids Sports


In [19]:
simscore = joined[['Original Donor ID','Project ID','Similarity Score']].reset_index()
simscore.columns = ['Other Donor ID', 'Original Donor ID','Project ID','Similarity Score']
simscore_df = pd.merge(simscore,projectitle,on=['Project ID'],how="inner")
simscore_df1 = simscore_df[['Original Donor ID','Other Donor ID','Project ID','Project Title','Similarity Score']]
simscore_df1.head()

,Original Donor ID,Other Donor ID,Project ID,Project Title,Similarity Score
0,0015dc157335e5ad6f569e5264bbf33e,006c2d57cca011d6040fed3a2ceb8179,dd19fb9ec77a69b6e9ffd956a3a88b5b,Mad About Mallets! Making Our Instruments Last,0.462709
1,0015dc157335e5ad6f569e5264bbf33e,009d5fc7b87883ffad248db5150bf1fc,022a1517e63f4e66845358f15ff7793e,Ocean Chemistry,0.379582
2,0015dc157335e5ad6f569e5264bbf33e,009d5fc7b87883ffad248db5150bf1fc,022a1517e63f4e66845358f15ff7793e,Ocean Chemistry,0.379582
3,0015dc157335e5ad6f569e5264bbf33e,009d5fc7b87883ffad248db5150bf1fc,022a1517e63f4e66845358f15ff7793e,Ocean Chemistry,0.379582
4,0015dc157335e5ad6f569e5264bbf33e,009d5fc7b87883ffad248db5150bf1fc,022a1517e63f4e66845358f15ff7793e,Ocean Chemistry,0.379582


In [20]:
def projrecommender(donorid):
    dataframe = simscore_df
    dataframe_subset = dataframe.loc[dataframe['Original Donor ID'] == donorid]
    simscore_mean = dataframe_subset.groupby(['Project ID','Project Title'],as_index=False).mean().sort_values('Similarity Score', ascending=False)
    simscore_df_final = simscore_mean[['Project Title','Similarity Score']]
    print(simscore_df_final.head(10))

In [21]:
projrecommender('0015dc157335e5ad6f569e5264bbf33e')

                                   Project Title  Similarity Score
852                               Traveling Tech          0.987247
2049                    iWrite, iDraw, iPublish!          0.942708
3281                 Who Wants to Play Kickball?          0.942708
2137               Chromebooks for Collaboration          0.942708
693                         Kindles for Learning          0.942337
787   High Interest Books For Struggling Readers          0.917215
2469                       Flexible and Engaged!          0.905215
3183                   Reading Takes You Places!          0.882871
565            Catering to Kids with Chromebooks          0.873391
1142                           Silence Is Golden          0.873240


In [34]:
def testrecommender(donoridlist):
    dataframe = simscore_df.copy()
    #datafdataframe_subset= rame_subset = dataframe.loc[dataframe['Original Donor ID'] == [i for i in donoridlist]]
    dataframe_subset = dataframe.loc[dataframe['Other Donor ID'].isin(donoridlist)]
    print(dataframe_subset.describe())
    simscore_mean = dataframe_subset.groupby('Project ID',as_index=False).mean().sort_values('Similarity Score', ascending=False)
    simscore_mean_more = simscore_mean[simscore_mean['Similarity Score'] > threshold]
    positive_list = simscore_mean_more['Project ID'].values.T.tolist()

    test_dataframe = testset3.copy()
    #test_dataframe_subset = test_dataframe.loc[test_dataframe['Donor ID'] == [i for i in donoridlist]]
    test_dataframe_subset = test_dataframe.loc[test_dataframe['Donor ID'].isin(donoridlist)]
    test_projectid_list = test_dataframe_subset['Project ID'].values.tolist()

    totals = test_dataframe_subset['Project ID'].count()
    print(totals)
    positive_count = len(set(positive_list).intersection(set(test_projectid_list)))

    print(totals)
    print(positive_count)

In [35]:
testrecommender(list_of_donor_ids)

       Similarity Score
count      89073.000000
mean           0.398987
std            0.174183
min            0.028002
25%            0.306358
50%            0.306358
75%            0.536364
max            0.987247
1248
1248
286
